# Нейросетевые модели поиска. Часть III. Векторный поиск.

# Домашнее задание

Цель домашнего задания - научиться строить индексы для векторного поиска на БОЛЬШИХ объемах данных, добиваясь при этом оптимального качества и скорости. Работать будем с уже известным датасетом MS MARCO (точнее, с сэмплом из него). Сэмпл генерируется следующим кодом:

In [ ]:
!pip install -r ./requirements.txt

import datasets
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
import string
import tensorflow_hub as hub

from IPython.display import clear_output
clear_output()

In [ ]:
msmarco_dataset = load_dataset("Tevatron/msmarco-passage")

In [ ]:
rows = []
i = -1
j = -1
for row in msmarco_dataset['train']:
    i += 1
    qid = i
    query = row['query']
    for pos_ex in row['positive_passages']:
        j += 1
        docid = j
        title = pos_ex['title']
        text = pos_ex['text']
        mark = 1
        rows.append([qid, query, docid, title, text, mark])
    for neg_ex in row['negative_passages']:
        j += 1
        docid = j
        title = neg_ex['title']
        text = neg_ex['text']
        mark = 0
        rows.append([qid, query, docid, title, text, mark])
df = pd.DataFrame(rows, columns=['qid', 'query', 'docid', 'title', 'text', 'mark'])

In [ ]:
df = df[:300000]
#df.to_csv('./data/homework_sample.csv', index=False)

In [ ]:
df

Баллы начисляются за выполнение следующих заданий:

# Предобработка данных [1 балл]

Возможно, прежде чем строить эмбеддинги из текстовых данных, стоит их предобработать? (использовать не только зону text, что-то сделать с пунктуацией, использовать зону не целиком). В случае использования только СЫРОЙ зоны text (как на семинаре) бал начислен не будет

In [ ]:
data = pd.read_csv('./data/homework_sample.csv')
#data = pd.read_csv('homework_sample.csv')

Объединяю title и text, удаляю пунктуацию, удаляю слова длиной меньше 4 символов (среди них артиклей и предлогов гораздо больше, чем важных для смысла), делю текст на блоки, из которых оставляю не все

In [ ]:
TEXT_BRANCHES = 4

rows = []
for ln in tqdm(data.to_numpy()):
    row = []
    row.append(int(ln[0]))
    row.append(ln[1])
    row.append(ln[2])
    
    text = str(ln[3]) + " " + ln[4]
    text.translate(str.maketrans(string.punctuation, " "*len(string.punctuation)))
    tokens = [s for s in text.split() if len(s) > 3]
    if len(tokens) == 0:
        row.append(" ")
    else:
        size = TEXT_BRANCHES * 2
        branch = 0
        while branch == 0:
            size //= 2
            branch = len(tokens) // size
        text = ""
        for i in range(0, size, 4):
            text = text + " " + " ".join(tokens[i * branch: (i + 1) * branch - 1])
        row.append(text)
    
    row.append(int(ln[5]))
    
    rows.append(row)
    
data_processed = pd.DataFrame(rows, columns=['qid', 'query', 'docid', 'text', 'mark'])

In [ ]:
#data_processed.to_csv('./data/homework_sample_processed.csv', index=False)

In [2]:
data_processed = pd.read_csv('./data/homework_sample_processed.csv').fillna(" ")
data_processed

,qid,query,docid,text,mark
0,0,where is whitemarsh island,0,"Whitemarsh Island, Georgia Whitemarsh Island,...",1
1,0,where is whitemarsh island,1,What military strategy used pacific? strategy...,0
2,0,where is whitemarsh island,2,Whakaari White Island island,0
3,0,where is whitemarsh island,3,"Jekyll Island Jekyll Island, 5,700 acres, sma...",0
4,0,where is whitemarsh island,4,Sibu Island Sibu Island. scuba diver Sibu,0
...,...,...,...,...,...
299995,9727,what does saddle soap do for boots,299995,Saddle Soap used clean smooth,1
299996,9727,what does saddle soap do for boots,299996,"Soap Recipes Making Homemade Soap with easy, ...",0
299997,9727,what does saddle soap do for boots,299997,Numbness Location: Back Into Bottom Foot Heel...,0
299998,9727,what does saddle soap do for boots,299998,Boot Fitting: Boots Supposed,0


In [15]:
relevant = {}
for ln in tqdm(data_processed.to_numpy()):
    relevant[ln[0]] = []
    if ln[4] == 1:
        relevant[ln[0]].append(ln[3])

100%|██████████| 300000/300000 [00:00<00:00, 755152.63it/s]


# Подбор нейронки [1 балл]

На семинаре для варки эмбеддингов использовался Universal Sentence Encoder. Возможно, стоит попробовать что-то другое? (балл будет начислен в том случае, если для варки индексов из следующих пунктов будет использована/ы другая/ие нейронки - да, использовать разные нейронки для разных индексов можно)

In [3]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Метрика для оценки качества [1 балл]

Нужно реализовать метрику для оценки качества индексов - ndcg@10

In [32]:
import torch
from torchmetrics.retrieval import RetrievalNormalizedDCG

def NDCG(preds, target, qids, k=10):
    ndcg = RetrievalNormalizedDCG(top_k=k)
    
    return ndcg(torch.Tensor(preds),
                torch.Tensor(target),
                indexes=torch.LongTensor(qids - min(qids)))

# Варка и инференс индексов [до 9 баллов]

В рамках этой части задания нужно сварить индексы 3 типов - annoy, faiss, hnswlib. Параметры можно выбирать любые (стоит ориентироваться на качество индекса, требуемую память и перф (см. ниже).)

За 1 индекс можно получить:

    1 балл,  если индекс состоит из  300000 документов
    2 балла, если индекс состоит из 1000000 документов
    3 балла, если индекс состоит из 2000000 документов

Важно:

1. Эмбеддинги, которые кладутся в индекс, должны быть сгенерированы нейронкой и соответствовать документам из датасета, иначе баллы аннулируются (сложить 2 миллиона эмбеддингов вида [1,1,1,...,1] нельзя)

2. Чтобы получить баллы за индекс, необходимо не только сварить, но и проинференсить его: прогнать на нем набор запросов из датасета (~65k), оценить качество результата (используя метрику ndcg@10), оценить время инференса (напр., так как в семинаре). Без этого баллы начислены не будут

3. При варке и инференсе индекса нельзя использовать больше 16GB оперативной памяти (иначе начислится 0 баллов)

In [ ]:
text_mat = []
for txt in tqdm(data_processed['text'].values):
    emb = embed([txt])
    text_mat.append(emb[0].numpy().tolist())
text_mat = np.array(text_mat)
"""with open('./embeddings/text_embs.npy', 'wb') as f:
    np.save(f, text_mat)"""

In [ ]:
query_mat = []
for txt in tqdm(data_processed['query'].unique()):
    emb = embed([txt])
    query_mat.append(emb[0].numpy().tolist())
query_mat = np.array(query_mat)
"""with open('./embeddings/query_embs.npy', 'wb') as f:
    np.save(f, query_mat)"""

In [4]:
with open('./embeddings/text_embs.npy', 'rb') as f:
    text_mat = np.load(f)
with open('./embeddings/query_embs.npy', 'rb') as f:
    query_mat = np.load(f)

### Annoy index

In [5]:
from annoy import AnnoyIndex

In [7]:
index = AnnoyIndex(f=512, metric='euclidean')
for i, vec in enumerate(tqdm(text_mat)):
    index.add_item(i, vec)

index.build(n_trees=500)
#index.save('./indexes/sample500.ann')

100%|██████████| 300000/300000 [00:38<00:00, 7780.74it/s]


True

In [ ]:
index = AnnoyIndex(f=512, metric='euclidean')
index.load('./indexes/sample500.ann')

In [39]:
def eval_qual(param=-1):
    predict = []
    qid = 0
    number = 0
    for ln in tqdm(data_processed.to_numpy()):
        if ln[0] == qid:
            number += 1
            continue
        
        items = index.get_nns_by_vector(vector=query_mat[qid], n=number, search_k=param)
        result = [int(i in relevant[qid]) for i in items]
        predict = predict + result
        
        number = 1
        qid = ln[0]
        
    items = index.get_nns_by_vector(vector=query_mat[qid], n=number, search_k=param)
    result = [int(i in relevant[qid]) for i in items]
    predict = predict + result
    
    
    return NDCG(predict, data_processed['mark'].values, data_processed['qid'].values)

In [40]:
%%time
eval_qual(500)

100%|██████████| 300000/300000 [00:21<00:00, 13966.35it/s]


CPU times: user 25.6 s, sys: 77.4 ms, total: 25.6 s
Wall time: 25.5 s


tensor(0.1511)

### hnswlib

In [46]:
import hnswlib

In [ ]:
emb_size = 512
index = faiss.IndexFlatL2(emb_size)
index.add(text_mat)
print(f" После добавления векторов: {index.ntotal}")

In [47]:
index = hnswlib.Index(space = 'l2', dim = 512)
index.init_index(max_elements = 300000, ef_construction = 20, M = 24)
index.add_items(text_mat)
index.set_ef(ef=50)

In [48]:
#index.save_index("./indexes/hnsw_efcons20_M24.bin")

In [49]:
index.load_index("./indexes/hnsw_efcons20_M24.bin")

In [75]:
def eval_qual():
    predict = []
    qid = 0
    number = 0
    for ln in tqdm(data_processed.to_numpy()):
        if ln[0] == qid:
            number += 1
            continue
        
        items, _ = index.knn_query(np.array([query_mat[qid]]).astype(np.float32), k=number)
        result = [int(i in relevant[qid]) for i in items[0]]
        predict = predict + result
        
        number = 1
        qid = ln[0]
        
    items, _ = index.knn_query(np.array([query_mat[qid]]).astype(np.float32), k=number)
    result = [int(i in relevant[qid]) for i in items[0]]
    predict = predict + result
    
    
    
    return NDCG(predict, data_processed['mark'].values, data_processed['qid'].values)

In [76]:
%%time
eval_qual()

100%|██████████| 300000/300000 [00:13<00:00, 22044.30it/s]


CPU times: user 17.5 s, sys: 78.2 ms, total: 17.6 s
Wall time: 17.6 s


tensor(0.1511)

# Соревнование [до 10 баллов]

Проводится среди индексов размерами 2000000 документов (остальные не оцениваются) в двух дисциплинах: по качеству (величина ndcg@10) и скорости (насколько быстро удалось прогнать полный набор запросов на этом индексе)

Оценивается только один индекс (указывается сдающим домашку) - если вы сварили 3 индекса по 2 миллиона доков каждый, то укажите, какой участвует в соревновании (напишите в ноутбуке). Иначе выбор будет сделан рандомно из доступных вариантов

Разбалловка:


За первое место по качеству: 5 баллов
    
За второе место по качеству: 3 балла
    
За все остальные места по качеству, кроме последнего: 1 балл

За первое место по скорости (при не последнем месте по качеству): 5 баллов

За второе место по скорости (при не последнем месте по качеству): 3 балла

За все остальные места по скорости (при не последнем месте по качеству), кроме последнего: 1 балл

# Процедура сдачи

**Вам надо:**

- Форкнуть эту репу;
- Создать бранч, в котором вы дальше будете работать;
- Выполнить все или часть заданий ноутбука;
- Запушить ваш бранч и поставить Pull Request.

Проверяющий счекаутит вашу бранчу и проверит работу.